In [ ]:
from sfcn import SFCN
import numpy as np
import pandas as pd
from volumedatagenerator import VolumeDataGeneratorRegression
import matplotlib.pyplot as plt

In [ ]:
name = 'sfcn_vanilla'
index=0

batch_size = 6
gpu_num = 6
cpu_workers = 8
epochs_num = 64

In [ ]:
train_df = pd.read_csv('csv/split_train.csv', index_col='eid').dropna()
valid_df = pd.read_csv('csv/split_valid.csv', index_col='eid').dropna()
test_df = pd.read_csv('csv/split_test.csv', index_col='eid').dropna()

In [ ]:
input_dim = [182, 218, 182]
num_output = len(train_df.columns)-1

In [ ]:
train_gen = VolumeDataGeneratorRegression(
    sample_df=train_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_preprocessing='quantile')

scaler_instance = train_gen.get_scaler_instance()

In [ ]:
valid_gen = VolumeDataGeneratorRegression(
    sample_df=valid_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False)

In [ ]:
test_gen = VolumeDataGeneratorRegression(
    sample_df=test_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False
)

In [ ]:
model = SFCN(
        input_dim=[182, 218, 182, 1], 
        output_dim=num_output,
        conv_num_filters=[32, 64, 128, 256, 256, 64], 
        conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
        conv_strides=[1, 1, 1, 1, 1, 1],
        conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
        pooling_size=[2, 2, 2, 2, 2],
        pooling_type=['max_pool', 'max_pool', 'max_pool', 'max_pool', 'max_pool'],
        batch_norm=True,
        dropout=False,
        softmax=False,
        gpu_num=gpu_num,
        name=name+'_'+str(index)
        )
model.compile(learning_rate=3e-4)

In [ ]:
model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=epochs_num, workers=cpu_workers)

In [ ]:
model.load_weights('weights/checkpoint_' + name + '_' + str(index))
model.evaluate_generator(valid_gen, batch_size, filename=name + '_val', workers=cpu_workers)
model.evaluate_generator(test_gen, batch_size, filename=name + '_test', workers=cpu_workers)